In [6]:
from astroquery import sdss

In [24]:
#Getting data from astropy

sdss_query = ''' SELECT TOP 10000
           p.modelMag_u, p.modelMag_g, p.modelMag_r, p.modelMag_i, p.modelMag_z,
           s.class, s.z
           FROM PhotoObjAll AS p JOIN specObjAll s ON s.bestobjid = p.objid
           WHERE p.mode = 1 AND s.sciencePrimary = 1 AND p.clean = 1 AND s.class != 'STAR'
           ORDER BY p.objid ASC
             '''
phot_z = sdss.SDSS.query_sql(sdss_query)

In [26]:
phot_z.to_pandas

<bound method Table.to_pandas of <Table length=10000>
modelMag_u modelMag_g modelMag_r modelMag_i modelMag_z class      z     
 float64    float64    float64    float64    float64   bytes6  float64  
---------- ---------- ---------- ---------- ---------- ------ ----------
  19.41061   18.23754   17.58132   17.20153   16.90159 GALAXY 0.03212454
  20.98014   20.28274   19.35493   18.85322   18.44382 GALAXY  0.2004682
  25.26718   22.31524   21.31751    19.9351   19.45918 GALAXY  0.7527065
  22.66974   22.00222   20.39819   19.57147   19.20231 GALAXY  0.5046716
  21.43851   18.80143    17.6947    17.1968    16.8061 GALAXY  0.1106111
  23.00731   21.34087   19.74872   19.14171   18.80783 GALAXY  0.2937785
  19.80807   18.33917   17.63962   17.24621       17.0 GALAXY 0.07429758
  19.54964   17.95799   17.02898     16.531   16.13408 GALAXY  0.1213151
  18.74425   17.37778   16.80538   16.51149   16.28756 GALAXY 0.04876465
  21.17984   19.97005   19.44376   19.11705   19.06998 GALAXY  0.04880

In [18]:
import astroquery
astroquery.__version__

'0.3.8'